In [149]:
import time
from selenium import webdriver
import requests
import shutil
from bs4 import BeautifulSoup
from selenium import webdriver
from base64 import decodestring, decodebytes
from selenium.webdriver.common.keys import Keys
 
import os
from urllib.parse import urlparse

quiz = "https://www.joe.ie/quiz/joe-friday-pub-quiz-week-254-727578"

driver = webdriver.Chrome('/Applications/chromedriver')
driver.get(quiz);
time.sleep(5) 

In [150]:
soup = BeautifulSoup(driver.page_source, 'lxml')
#driver.find_element_by_xpath("//div[@data-wpvq-answer='33419']").click()
xpath = "//div[@data-wpvq-answer="

In [151]:
questions = soup.find_all("div", class_="wpvq-question-label")
answers = soup.find_all("label", class_="vq-css-label")
checks = soup.find_all("input", class_="vq-css-checkbox")
count = 0
ans = []
qs = {}
qq = []
for q in questions:
    relanswers = answers[count*3: (count*3)+3]
    atts = checks[count*3].attrs
    path = xpath+atts['data-wpvq-answer']+"]"
    #print(atts)
    driver.find_element_by_xpath(path).click()
    time.sleep(1)
    for cho in relanswers:
        ans.append(cho.text.strip('\n'))
    count = count +1
    qs['Question'] = q.text
    qs['Answers'] = ans
    ans=[]
    qq.append(qs)
    qs = {}



In [152]:
import pandas as pd
soup = BeautifulSoup(driver.page_source, 'lxml')
rechecks = soup.find_all("div", class_="wpvq-answer")

corrected = []
counter = 0
corr = []
for i in range(0,len(questions)):
    for g in range(i*3,(i*3)+3):
        res = rechecks[g]
        dic = res.attrs
        #print (dic['class'])
        corr.append(dic['class'])
    corrected.append(corr)
    corr = []

def checker(x):
    possible = x['Answers']
    resulst = x['results']
    response = ''
    for i in range(0,len(possible)):
        if "wpvq-answer-true" in resulst[i]:
            response = possible[i]
    return response
    

df = pd.DataFrame(qq)
df['results'] = corrected
df['answer'] = df.apply(lambda x: checker(x), axis=1)
df.to_csv('joe.csv')

In [153]:
driver.quit()


In [154]:
df[['Question','answer']]

,Question,answer
0,ROUND ONE: General Knowledge. How many countri...,27
1,What is the Guinness World Record for the olde...,122 years
2,Which of these social media platforms was laun...,Snapchat
3,What vegetable do Americans call 'zucchini'?,Courgette
4,How many planets make up our solar system?,8
5,ROUND TWO: Sport. What is the name of the new ...,The Hundred
6,The Sheffield Steelers and the Nottingham Pant...,Ice hockey
7,How many London-based teams will be taking par...,7
8,Where were the last Winter Olympics held?,"Pyeongchang, South Korea"
9,Which country are the British and Irish Lions ...,South Africa
